In [1]:
from pprint import pprint
from pathlib import Path
from dotenv import load_dotenv

# load openai api key
load_dotenv(Path().cwd().parent / '.env')

True

In [7]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

In [8]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

DB_NAME='nation'
DB_HOST='127.0.0.1:3306'
DB_USER='root'
DB_PASSWORD='rootpass'

conn_uri = f'mariadb+mariadbconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}'
db = SQLDatabase.from_uri(conn_uri)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

executer = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [11]:
executer.run("list all the tables in the database?")



> Entering new  chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


Action: sql_db_list_tables
Action Input: ""
Observation: continents, countries, country_languages, country_stats, guests, languages, region_areas, regions, vips
Thought:I can see that there are several tables in the database. Now I need to query the schema of the relevant tables to see what columns are available.
Action: sql_db_schema
Action Input: "countries, country_stats"
Observation: 
CREATE TABLE countries (
	country_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	name VARCHAR(50), 
	area DECIMAL(10, 2) NOT NULL, 
	national_day DATE, 
	country_code2 CHAR(2) NOT NULL, 
	country_code3 CHAR(3) NOT NULL, 
	region_id INTEGER(11) NOT NULL, 
	PRIMARY KEY (country_id), 
	CONSTRAINT countries_ibfk_1 FOREIGN KEY(region_id) REFERENCES regions (region_id)
)DEFAULT CHARSET=utf8mb3 ENGINE=InnoDB COLLATE utf8mb3_general_ci

/*
3 rows from countries table:
country_id	name	area	national_day	country_code2	country_code3	region_id
1	Aruba	193.00	None	AW	ABW	1
2	Afghanistan	652090.00	1919-08-19	AF	AFG	2
3	A

'The tables in the database are "continents", "countries", "country_languages", "country_stats", "guests", "languages", "region_areas", "regions", and "vips".'

In [9]:
executer.run("How many languages there is?")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: continents, countries, country_languages, country_stats, guests, languages, region_areas, regions, vips
Thought:I can query the "languages" table to find the number of languages.
Action: sql_db_schema
Action Input: "languages"
Observation: 
CREATE TABLE languages (
	language_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	language VARCHAR(50) NOT NULL, 
	PRIMARY KEY (language_id)
)DEFAULT CHARSET=utf8mb3 ENGINE=InnoDB COLLATE utf8mb3_general_ci

/*
3 rows from languages table:
language_id	language
1	Dutch
2	English
3	Papiamento
*/
Thought:I can use the "languages" table to count the number of languages.
Action: sql_db_query
Action Input: "SELECT COUNT(*) FROM languages"
Observation: [(457,)]
Thought:I now know the final answer.
Final Answer: There are 457 languages.

> Finished chain.


'There are 457 languages.'

In [10]:
executer.run("What are the spoken languages of the top 5 GDP countries?")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: continents, countries, country_languages, country_stats, guests, languages, region_areas, regions, vips
Thought:I can query the "countries" and "languages" tables to find the spoken languages of the top GDP countries.
Action: sql_db_schema
Action Input: "countries, languages"
Observation: 
CREATE TABLE countries (
	country_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	name VARCHAR(50), 
	area DECIMAL(10, 2) NOT NULL, 
	national_day DATE, 
	country_code2 CHAR(2) NOT NULL, 
	country_code3 CHAR(3) NOT NULL, 
	region_id INTEGER(11) NOT NULL, 
	PRIMARY KEY (country_id), 
	CONSTRAINT countries_ibfk_1 FOREIGN KEY(region_id) REFERENCES regions (region_id)
)DEFAULT CHARSET=utf8mb3 ENGINE=InnoDB COLLATE utf8mb3_general_ci

/*
3 rows from countries table:
country_id	name	area	national_day	country_code2	country_code3	region_id
1	Aruba	193.00	None	AW	ABW	1
2	Afghanistan	652090.00	1919-08-19	AF	AFG	2
3	Angola	1246700.00	

'Russian, Belorussian, Ukrainian, Kazakh, Tatar'